In [2]:
#important imports
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import spacy
import numpy as np
from spacy.en import English
parser = English()

## ENTER PASSAGE

In [74]:
#Opening the passage File
file = open("havard1.txt")
text = file.read()


## ENTER QUERY

In [ ]:
query_question = "When did the undergraduate program become coeducational?"

## process

In [4]:
#Stopwords including the punctuation marks.
stopword =stopwords.words('english')
punctuation_list = ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']
question_list = ["How","What","Where","Why","Which","Who", "When"]
stopword = stopword + punctuation_list + question_list

In [54]:
#Query question
lemmatizer = WordNetLemmatizer()

query_words = word_tokenize(query_question)
query_words_lower = ([x.lower() for x in query_words])
query_words_lower =  ([word for word in query_words_lower if word not in stopword])
query_words_lower = [lemmatizer.lemmatize(x) for x in query_words_lower]
print(query_words_lower)


['undergraduate', 'program', 'become', 'coeducational']


In [55]:
#Tokenizing the passage into sentences
passage_sentences = sent_tokenize(text)

In [56]:
count_list = []
lemmatizer = WordNetLemmatizer()
for sentence in passage_sentences:
    count = 0
    words = word_tokenize(sentence)
    words = ([x.lower() for x in words])
    words = ([x for x in words if x not in stopword])
    words = [lemmatizer.lemmatize(x) for x in words]
    # Finding the actual sentence where answer is
    for x in query_words_lower:
        if (x in words):
            count = count+1
    count_list.append(count)    

In [57]:
# Finding the sentence where the sentence exists
index_of_sentence = count_list.index(max(count_list))
answer_sentence = passage_sentences[index_of_sentence]

In [58]:
#Finding the POS tags in the query question
nlp = spacy.load('en')
doc = nlp(query_question)
for word in doc:
    print(word.text, word.tag_, word.pos_)

When WRB ADV
did VBD VERB
the DT DET
undergraduate JJ ADJ
program NN NOUN
become VB VERB
coeducational JJ ADJ
? . PUNCT


In [59]:
ans = nlp(passage_sentences[index_of_sentence])
ans

The undergraduate college became coeducational after its 1977 merger with Radcliffe College.

In [60]:
wh_map_answer = {
    "who": ["PERSON", "ORG"],
    "when": ["DATE", "TIME"],
    "where": ["GPE"],
    "which": ["PERSON", "ORG", "GPE"],
    "what": ["PERSON", "ORG"],
    "default": ["PERSON", "ORG", "GPE", "TIME", "DATE"]
}

In [61]:
words_in_question = word_tokenize(query_question)
words_in_question = ([x.lower() for x in words_in_question])
intersection_words = list(set(words_in_question) & set(wh_map_answer.keys()))
wh_type = intersection_words[0] if intersection_words else "default"
answer = [(ent, ent.label_) for ent in ans.ents if ent.label_ in wh_map_answer[wh_type]]
print(query_question)
print(answer)

When did the undergraduate program become coeducational?
[(1977, 'DATE')]


In [62]:
# Experiments starts here
query_words =  ([word for word in query_words if word not in stopword])
query_words = [lemmatizer.lemmatize(x) for x in query_words]
query_words

['undergraduate', 'program', 'become', 'coeducational']

In [63]:
#coming down to one answer
answer_candidates = [str(ent) for ent in ans.ents if ent.label_ in wh_map_answer[wh_type]]
print(list(answer_candidates))

['1977']


In [64]:
answer_candidates = list(answer_candidates)

In [65]:
for i in query_words:
    if(i in str(answer_candidates)):
        answer_candidates.remove(i)

In [66]:
answer_tokenise = nltk.word_tokenize(answer_sentence)

In [67]:
dist = np.zeros((len(query_words), len(answer_candidates)))

In [68]:
dist.shape

(4, 1)

In [69]:
i=0
for ac in answer_candidates:
    min = 0
#     ac = str(ac).split()[0]
    j = 0
    for qw in query_words:
#         qw = str(qw).split()[0]
        if (qw in answer_tokenise):
#             tmp = abs(answer_tokenise.index(ac) - answer_tokenise.index(qw))
            tmp = abs(answer_sentence.find(str(ac))-answer_sentence.find(str(qw)))
            print (j,"----", i)
            print (qw,"------" , ac)
            dist[j][i]+=tmp
        j=j+1
    i=i+1

0 ---- 0
undergraduate ------ 1977
3 ---- 0
coeducational ------ 1977


In [70]:
dist

array([[ 53.],
       [  0.],
       [  0.],
       [ 24.]])

In [71]:
a = dist.sum(axis = 0)

In [72]:
m = a.argmin()

In [75]:
print(query_question)
print(answer_candidates[m])

When did the undergraduate program become coeducational?
1977
